In [20]:
from npy_append_array import NpyAppendArray
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import sys
import os

notebook_path = os.path.abspath('')
project_root = os.path.dirname(notebook_path)
sys.path.append(project_root)

from helpers.read_osc_to_df import *
#from modules.coincidence_filter import *
from modules.filters import *



ModuleNotFoundError: No module named 'mimocorb'

In [21]:
def tag_pulses(input_data):   
    """find all valid pulses 

    This function to be called by instance of class mimoCoRB.rbTransfer

        Args:  input data as structured ndarray

        Returns: list of parameterized pulses
        
        
        Currently only works with two channels. Every part which requires two channels ist marked with ### two
    """

    # Find all the peaks and store them in a dictionary
    peaks, peaks_prop = tag_peaks(input_data, peak_minimal_prominence, peak_minimal_distance, peak_minimal_width)

    peak_data= np.zeros( (1,), dtype=pulse_par_dtype)
    
    # Only consider events with one peak in the trigger_channel
    if len(peaks[trigger_channel]!=1):
        return None
    trigger_peak=peaks[trigger_channel][0]
    for signal_channel in signal_channels:
        if len(peaks[signal_channel])!=1: ### two
            return None ### two
        Delta_T=coincidence_window[signal_channel]['offest_from_trigger']/sample_time_ns
        w=coincidence_window[signal_channel]['width_of_window']/sample_time_ns
        for peak in peaks[signal_channel]:
            if -w/2<=peak-Delta_T-trigger_peak<=w/2:
                # explanation: t+Delta_T-w/2<=peak<=t+Delta_T+w/2
                """ab hier erstmal nur für einen trigger und einen signal channel"""
                peak_data[0][signal_channel+'_position'] = peak
                peak_data[0][trigger_channel+'_position'] = trigger_peak
                peak_data[0][signal_channel+'_height'] = peaks_prop[signal_channel]['relative_height']
                peak_data[0][trigger_channel+'_height'] = peaks_prop[trigger_channel]['relative_height']
    
    
    return [peak_data]  

In [22]:
messung="1"

#later to be imported from config_dict
sample_time_ns = 256
analogue_offset = 0*1000
trigger_channel = 'ch2'
coincid_channel = 'ch1'

peak_minimal_prominence = {'ch1': 200, 'ch2': 200}
peak_minimal_distance = {'ch1': 400, 'ch2': 400}
peak_minimal_width = {'ch1': 10, 'ch2': 10}

gradient_lower_bound = {'ch1': 0, 'ch2': 0}
gradient_upper_bound = {'ch1': 0, 'ch2': 0}

number_of_samples=1000

window_us=8
window=window_us*1000/sample_time_ns

color_table={'ch1': "tab:orange", 'ch2': "tab:blue"}
channel_names=np.array(['ch1','ch2'])

In [23]:
data,number_of_triggers=import_data(project_root+ "/measurements/measurement_3_" + messung +"min.npy",number_of_samples, channel_names)


Data read successfully, recorded triggers = 2752


In [24]:
for d in data[0:5]:
    peaks,peaks_prop=tag_peaks(d,peak_minimal_prominence, peak_minimal_distance, peak_minimal_width, gradient_upper_bound, gradient_lower_bound)
    #print(peaks_prop['ch2'])
    for key in d.dtype.names:
        plt.plot(d[key],color=color_table[key])
        plt.vlines(peaks_prop[key]['start_position'],ymin=peaks_prop[key]['absolute_height_at_start'],ymax=peaks_prop[key]['absolute_height_at_peak'],colors=color_table[key])
        plt.hlines(peaks_prop[key]['absolute_height_at_peak'],xmin=peaks_prop[key]['start_position'],xmax=peaks[key],colors=color_table[key])
    plt.show()

NameError: name 'tag_peaks' is not defined

In [5]:
print(data[0].dtype)

[('ch1', '<i2', (1000,)), ('ch2', '<i2', (1000,))]


In [6]:
peak_data=np.zeros( (1,), dtype=('ch'))

signal_channel='ch1'
trigger_channel='ch2'
peak=10
trigger_peak=11
peaks_prop={signal_channel: {'relative_height': 20},trigger_channel: {'relative_height':21}}

print(peak_data)

peak_data[0][signal_channel+'_position'] = peak
peak_data[0][trigger_channel+'_position'] = trigger_peak
peak_data[0][signal_channel+'_height'] = peaks_prop[signal_channel]['relative_height']
peak_data[0][trigger_channel+'_height'] = peaks_prop[trigger_channel]['relative_height']

TypeError: data type 'ch' not understood